In [1]:
import os
from openai import OpenAI
client = OpenAI()

client.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
assistant = client.beta.assistants.create(
    name="수학 선생님",
    instructions="당신은 친절한 수학선생님입니다. 사용자가 질문하는 수학문제에 대답하기 위해 code를 작성하고 실행하세요.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-3.5-turbo"
)
assistant

Assistant(id='asst_QKBO6sa8HCqtqnTz6BqebjeM', created_at=1709866222, description=None, file_ids=[], instructions='당신은 친절한 수학선생님입니다. 사용자가 질문하는 수학문제에 대답하기 위해 code를 작성하고 실행하세요.', metadata={}, model='gpt-3.5-turbo', name='수학 선생님', object='assistant', tools=[ToolCodeInterpreter(type='code_interpreter')])

In [3]:
thread = client.beta.threads.create()
thread

Thread(id='thread_vhMsRxfsHs1hWczY9RZQXkO5', created_at=1709866223, metadata={}, object='thread')

In [4]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="`3x + 11 = 14`라는 방정식을 풀려고 하는데 도와줄 수 있어?"
)

In [5]:
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.data)

[ThreadMessage(id='msg_3l4bqko7CYGyTQyYnktIghlH', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='`3x + 11 = 14`라는 방정식을 풀려고 하는데 도와줄 수 있어?'), type='text')], created_at=1709866223, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_vhMsRxfsHs1hWczY9RZQXkO5')]


In [6]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="수학문제에 대해 설명하고 답을 같이 말해줘."
)

In [7]:
import time

while run.status != "completed"  :
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    time.sleep(0.1)

In [8]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [9]:
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.data)

[ThreadMessage(id='msg_ikaJnjLyHdoCF0B7zxmAnD2R', assistant_id='asst_QKBO6sa8HCqtqnTz6BqebjeM', content=[MessageContentText(text=Text(annotations=[], value='주어진 방정식을 해결한 결과, x의 값은 1입니다. 따라서 주어진 방정식의 해는 x = 1입니다.'), type='text')], created_at=1709866229, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_RnrzVqtTduGWnAawf0boHkQA', thread_id='thread_vhMsRxfsHs1hWczY9RZQXkO5'), ThreadMessage(id='msg_KZKuTd9I3pfY1Z8ib3fHZC3D', assistant_id='asst_QKBO6sa8HCqtqnTz6BqebjeM', content=[MessageContentText(text=Text(annotations=[], value='물론이죠! 이 방정식을 풀기 위해 단계별로 진행하겠습니다.\n\n주어진 방정식은 다음과 같습니다:\n\\[ 3x + 11 = 14 \\]\n\n1. 먼저 상수항을 한 쪽으로 옮겨주겠습니다. \n   \\[ 3x = 14 - 11 \\]\n\n2. 그 다음으로 x에 대해 isolate 해줄 것입니다.\n   \\[ x = \\frac{14 - 11}{3} \\]\n\n이제 위 단계를 계산하여 x의 값을 찾아보겠습니다.'), type='text')], created_at=1709866224, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_RnrzVqtTduGWnAawf0boHkQA', thread_id='thread_vhMsRxfsHs1hWczY9RZQXkO5'), ThreadM